In [1]:
import pandas as pd
import numpy as np
import talib as ta
from pycoingecko import CoinGeckoAPI
from datetime import datetime as dt, timedelta as td

In [2]:
config = {
    'id': 'bitcoin',
    'vs_curr': 'usd',
    'range': 24, # max 24
    'save_csv_path': 'datasets/crypto_ta_btc_02.csv'
}

In [3]:
now = dt.strptime(dt.now().strftime("%Y-%m-%d %H:00:00"),"%Y-%m-%d %H:00:00")
timestamp_list = [{'from': dt.timestamp(now - td(days=(x+1)*90)), 'to': dt.timestamp(now - td(days=x*90))} for x in reversed(range(config['range']))]

print(timestamp_list)

[{'from': 1444485600.0, 'to': 1452265200.0}, {'from': 1452265200.0, 'to': 1460037600.0}, {'from': 1460037600.0, 'to': 1467813600.0}, {'from': 1467813600.0, 'to': 1475589600.0}, {'from': 1475589600.0, 'to': 1483369200.0}, {'from': 1483369200.0, 'to': 1491141600.0}, {'from': 1491141600.0, 'to': 1498917600.0}, {'from': 1498917600.0, 'to': 1506693600.0}, {'from': 1506693600.0, 'to': 1514473200.0}, {'from': 1514473200.0, 'to': 1522245600.0}, {'from': 1522245600.0, 'to': 1530021600.0}, {'from': 1530021600.0, 'to': 1537797600.0}, {'from': 1537797600.0, 'to': 1545577200.0}, {'from': 1545577200.0, 'to': 1553353200.0}, {'from': 1553353200.0, 'to': 1561125600.0}, {'from': 1561125600.0, 'to': 1568901600.0}, {'from': 1568901600.0, 'to': 1576681200.0}, {'from': 1576681200.0, 'to': 1584457200.0}, {'from': 1584457200.0, 'to': 1592229600.0}, {'from': 1592229600.0, 'to': 1600005600.0}, {'from': 1600005600.0, 'to': 1607785200.0}, {'from': 1607785200.0, 'to': 1615561200.0}, {'from': 1615561200.0, 'to': 16

In [4]:
cg = CoinGeckoAPI()
temp_closes = []
temp_volumes = []

bitcoin_close = pd.read_csv('bitcoin_close_full.csv')

# for timestamp in timestamp_list:
#     mk_chart = cg.get_coin_market_chart_range_by_id(id=config['id'], vs_currency=config['vs_curr'], from_timestamp=timestamp['from'], to_timestamp=timestamp['to'])
#     for i in range(len(mk_chart['prices'])):
#         temp_closes.append(mk_chart['prices'][i][1])
#         temp_volumes.append(mk_chart['total_volumes'][i][1])

closes = np.array(bitcoin_close['closes'])
volumes = np.array(bitcoin_close['volumes'])

print(closes)
print(volumes)

[  245.3209       247.5392       245.5661     ... 52643.0475473
 52592.264954   52693.78878816]
[8.91650697e+07 7.92682217e+07 1.08125414e+08 ... 3.90660348e+10
 3.96247676e+10 3.98123294e+10]


In [5]:
sma_200 = ta.SMA(closes, 200)
ema_8 = ta.EMA(closes, 8)
ema_20 = ta.EMA(closes, 20)
ema_25 = ta.EMA(closes, 25)
ema_30 = ta.EMA(closes, 30)
ema_35 = ta.EMA(closes, 35)
ema_40 = ta.EMA(closes, 40)
ema_45 = ta.EMA(closes, 45)
ema_50 = ta.EMA(closes, 50)
sma_vol = ta.SMA(volumes, 20)
bbands_up, bbands_mid, bbands_low = ta.BBANDS(closes, timeperiod=20, nbdevup=2, nbdevdn=2, matype=0)
cmo = ta.CMO(closes, timeperiod=14)
macd, macdsignal, macdhist = ta.MACD(closes, fastperiod=12, slowperiod=26, signalperiod=9)
macdhist = ta.ROC(macdhist, timeperiod=1)
ppo = ta.PPO(closes, fastperiod=12, slowperiod=26, matype=0)
linearreg = ta.LINEARREG(closes, timeperiod=14)
tsf = ta.TSF(closes, timeperiod=14)

In [12]:
class_data = []
roc = ta.ROC(closes, timeperiod=1)

for n in range(0,len(roc)):
    if n + 2 < len(roc):
        if roc[n] < 0 and roc[n+1] < 0 and roc[n+2] < 0:
            class_data.append('LOW')
        elif roc[n] > 0 and roc[n+1] > 0 and roc[n+2] > 0:
            class_data.append('HIGH')
        else:
            class_data.append('NEUTRAL')
            
print(len(class_data))

22908


In [13]:
# orig_dt = pd.DataFrame({'closes': closes, 'volumes': volumes})
dt = pd.DataFrame({
    'price': closes[200:-3],
    'sma_200': sma_200[200:-3],
    'ema_8': ema_8[200:-3],
    'ema_20': ema_20[200:-3],
    'ema_25': ema_25[200:-3],
    'ema_30': ema_30[200:-3],
    'ema_35': ema_35[200:-3],
    'ema_40': ema_40[200:-3],
    'ema_45': ema_45[200:-3],
    'ema_50': ema_50[200:-3],
    'sma_vol': sma_vol[200:-3],
    'bbands_up': bbands_up[200:-3],
    'bbands_mid': bbands_mid[200:-3],
    'bbands_low': bbands_low[200:-3],
    'cmo': cmo[200:-3],
    'macd': macd[200:-3],
    'macdsignal': macdsignal[200:-3],
    'macdhist': macdhist[200:-3],
    'ppo': ppo[200:-3],
    'linearreg': linearreg[200:-3],
    'tsf': tsf[200:-3],
    'trend': np.array(class_data[201::]) # tirado 2 valores do final
})

dt.describe()
print(dt)

              price       sma_200         ema_8        ema_20        ema_25  \
0        444.789000    391.802118    451.280749    440.756688    437.772289   
1        449.381000    392.811327    450.858583    441.578051    438.665267   
2        456.566000    393.866327    452.126898    443.005475    440.042247   
3        447.903000    394.858816    451.188254    443.471906    440.646920   
4        452.592000    395.858298    451.500197    444.340486    441.565772   
...             ...           ...           ...           ...           ...   
22702  52503.990058  49336.460548  51947.661405  51682.754196  51584.076441   
22703  52739.800166  49356.476479  52123.692241  51783.425241  51672.978266   
22704  52449.901543  49374.585791  52196.183197  51846.899174  51732.741595   
22705  52506.302531  49393.184464  52265.098604  51909.699494  51792.246282   
22706  52774.412756  49412.868820  52378.279527  51992.053138  51867.797549   

             ema_30        ema_35        ema_40    

In [14]:
dt.to_csv(config['save_csv_path'], index=False)
# orig_dt.to_csv('bitcoin_close_full.csv', index=False)